# Example of fitting: 51 Pegasi b

To show how to fit some RV data, let's start with possibly the most famous exoplanet, 51 Peg b. We will quickly fit a circular model, find a best-fit solution using MAP, explore parameter uncertainty with MAP, and derive a mass-estimate $M_p\sin{i}$.

The ELODIE data and parameters used in this example notebook for 51 Peg b were obtained from [Birkby et al. 2017](http://doi.org/10.3847/1538-3881/aa5c87).

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from astropy.stats import knuth_bin_width

import ravest.prior
from ravest.fit import Fitter
from ravest.model import calculate_mpsini
from ravest.param import Parameter, Parameterisation

Import and inspect the data

In [ ]:
data = pd.read_csv('example_data/51Pegb.txt', delimiter='\s+', )
data

In [ ]:
plt.figure(figsize=(15,3.5))
plt.title("51 Peg b ELODIE data")
plt.ylabel("Radial Velocity [m/s]")
plt.xlabel("BJD_TDB")
plt.errorbar(data["time"], data["vel"], yerr=data["verr"], marker=".", linestyle="None")
plt.show()

In [ ]:
fitter = Fitter(planet_letters=["b"], parameterisation=Parameterisation("P K e w Tc"))
fitter.add_data(time=data["time"].to_numpy(), 
                vel=data["vel"].to_numpy(), 
                verr=data["verr"].to_numpy(), 
                t0=np.mean(data["time"]))
print("t0:", fitter.t0)

# Construct the params dict
# These values will be used as your initial guess for the MAP fit
params_dict = {
          "P_b": Parameter(4.23, "d", fixed=False),
          "K_b": Parameter(60, "m/s", fixed=False),
          "e_b": Parameter(0, "", fixed=True),
          "w_b": Parameter(np.pi/2, "rad", fixed=True),
          "Tc_b": Parameter(2456325.94, "d", fixed=False),

          "g": Parameter(np.median(fitter.vel), "m/s", fixed=False),
          "gd": Parameter(0, "m/s/day", fixed=True),
          "gdd": Parameter(0, "m/s/day^2", fixed=True),
          "jit": Parameter(0, "m/s", fixed=True),}

fitter.params = params_dict
fitter.params

Next, we need to add prior functions for the free parameters. Ravest has a range of prior functions available:

In [ ]:
ravest.prior.PRIOR_FUNCTIONS

In [ ]:
# Construct the priors dict. Every parameter that isn't fixed requires a prior.
priors_dict = {
          "P_b": ravest.prior.Normal(params_dict["P_b"].value, 0.00001),
          "K_b": ravest.prior.Uniform(lower=0, upper=100),
          "Tc_b": ravest.prior.Uniform(lower=params_dict["Tc_b"].value-2.0, upper=params_dict["Tc_b"].value+2.0),
          "g": ravest.prior.Uniform(lower=params_dict["g"].value-np.std(fitter.vel), upper=params_dict["g"].value+np.std(fitter.vel)),
        }

fitter.priors = priors_dict
fitter.priors

Now that we have loaded the `Fitter` with the data, our parameterisation, our initial parameter values, and priors for each of the free parameters, we can now fit the free parameters of the model to the data. First, Maximum A Posteriori (MAP) optimisation is performed to find the best-fit solution that maximises the log-probability (log likelihood + log priors).

In [ ]:
map_result = fitter.find_map_estimate(method="Powell")
map_result

In [ ]:
fitter.plot_MAP_rv(map_result=map_result, save=False)

The residuals look okay, but because of how spread out the data is over time, it's hard to really gauge how well the planet Keplerian model is fitting to the data. So let's fold the data over the orbital period, and create an orbital phase plot instead, showing just the RV contribution of the planet (subtracting any system trend).

In [ ]:
fitter.plot_MAP_phase(map_result=map_result, planet_letter="b", save=False)

That looks like a reasonable fit!

Next, MCMC is used to explore the parameter space to estimate the parameter uncertainties and fully explore the parameter space. It's a good idea to ensure your MCMC walkers don't all initialise in the same location - to do this, the function`generate_initial_walker_positions` below ensures each walker starts in a random position that still satisfies the boundaries of the prior functions and that is astrophysically valid (e.g. still satifies $P>0$ or $0\leq{e}<1$.)

In [ ]:
nwalkers = 2 * len(fitter.free_params_dict)
mcmc_init = fitter.generate_initial_walker_positions(nwalkers=nwalkers, verbose=True)

In [ ]:
nsteps = 25000

# Fit the free parameters to the data
samples = fitter.run_mcmc(initial_positions=mcmc_init, nwalkers=nwalkers, nsteps=nsteps, progress=True)  # This will take a while!

Now that the MCMC is finished, the `emcee` sampler has been saved into the `Fitter` object. We can therefore interact with it in the usual way to export the samples, as a numpy array that can be passed into other functions (such as for comparing two models by calculating the Bayesian evidence - example notebook coming soon!). We can also export them into a Pandas dataframe, which keeps each parameter labelled. In both cases, we can pass in the `discard_start` and `thin` arguments as desired.

In [ ]:
# Get the results from the sampler, as a numpy array. Discard the burn-in phase.
samples = fitter.get_samples_np(discard_start=500)
print(samples.shape)  # (nsteps, nwalkers, nparams)

# Get the (flattened) samples as a labelled Pandas dataframe
samples_df = fitter.get_samples_df(discard_start=500, thin=1)
samples_df  # shape (nsteps*nwalkers, nparams)

To inspect the chains visually, we can plot (and optionally save) the time series of each parameter in the chain.

In [ ]:
fitter.plot_chains(discard_start=500, save=False)

We can also visualise (and optionally save) the posterior parameter distributions in corner plots, using the `corner` module.

In [ ]:
fitter.plot_corner(discard_start=500, save=False)

In this case, the automatically generated value and error (the 16th, 50th and 84th percentiles) are fairly representative, but you should always inspect the posterior corner plots. For further analysis and inspection, recall we can get a dataframe of the samples, e.g. to plot them in a histogram, with the `Fitter.get_samples_df()` method we saw earlier.

We can also look at the posterior RV to eyeball whether it is a good fit to the data. (We use `thin=10` to generate the plot faster)

In [ ]:
fitter.plot_posterior_rv(discard_start=500, thin=10, save=False)
fitter.plot_posterior_phase(planet_letter="b", discard_start=500, thin=10, save=False)

## Calculate $M_p\sin{i}$
Now that we have fitted for the parameters, we can investigate the $M_p\sin{i}$ of 51 Peg b. To do this, we'll pass in the samples from the `Fitter` for the parameters we need. We also need the stellar mass, which I've again obtained from Birkby et al. 2017. The relationship between planetary mass and RV semi-amplitude is given by

$$ M_p\sin{i}=K\sqrt{1-e^2}\left(\frac{PM^2_*}{2\pi G}\right)^{1/3}. $$

In [ ]:
# Stellar mass [solar masses] from Birkby et al. 2017.
mass_star_val = 1.11
mass_star_err = 0.066


# Create a distribution to ensure the uncertainty in the stellar mass is captured in the mpsini uncertainty
np.random.seed(47)  # For reproducibility
mass_star = np.random.normal(loc=mass_star_val, scale=mass_star_err, size=len(samples_df))
# Ensure all values in mass_star are positive (incredibly unlikely to be an issue with these values, but just in case)
while any(mass_star <= 0):
    mass_star[mass_star <= 0] = np.random.normal(loc=mass_star_val, scale=mass_star_err, size=sum(mass_star <= 0))

# Get the posterior samples (both free and fixed) as a dictionary
post_samples = fitter.get_posterior_params_dict(discard_start=500, thin=1)

# Calculate the mpsini value - choose whether M_jupiter, M_earth or kg
mpsini = calculate_mpsini(mass_star, post_samples["P_b"], post_samples["K_b"], post_samples["e_b"], unit="M_jupiter")

# Calculate the bin width using Knuth bin width
width, edges = knuth_bin_width(mpsini, return_bins=True)

# Let's plot the mpsini posterior distribution in a histogram
fig, ax = plt.subplots(1, 1)
ax.set_title("Minimum mass estimate $M_p\sin{i}$ for 51 Peg b")
ax.set_xlabel("$M_p \sin(i)$ [M$_J$]")
ax.set_ylabel("Frequency")
plot = ax.hist(mpsini, bins=edges, color="tab:blue", alpha=0.7)

# Let's overplot the 16th, 50th and 84th percentiles
ps = np.percentile(mpsini, [16, 50, 84])
# Search for the heights of the bins in which the percentiles are located
heights = plot[0][np.searchsorted(plot[1], ps, side='left')-1]
# The line height will be bin-height / y_bound
_, ymax = ax.get_ybound()
ax.axvline(ps[0], label='16%', color='blue', linestyle=':', linewidth=2, ymax=heights[0] / ymax)
ax.axvline(ps[1], label='50%', color='blue', linestyle='--', linewidth=2, ymax=heights[1] / ymax)
ax.axvline(ps[2], label='84%', color='blue', linestyle=':', linewidth=2, ymax=heights[2] / ymax)

# print the mass to two decimal places with +- uncertaintiees (the 16 84 percentiles) in a box in top left
ax.text(0.02, 0.97, f"$M_{{p}}\sin{{i}}$: {ps[1]:.2f} +{ps[2]-ps[1]:.2f} -{ps[1]-ps[0]:.2f} $M_{{J}}$", transform=ax.transAxes, fontsize=10, va='top', ha='left', bbox=dict(boxstyle='round', facecolor='white', edgecolor='gray', alpha=0.8))
plt.legend(loc="upper right")
print(f"51 peg b Mpsini: {ps[1]:} +{ps[2]-ps[1]:.1g} -{ps[1]-ps[0]:.1g} M_jupiter")